In [9]:
!pip install openai==1.35.3
!pip install pyautogen==0.2.31
!pip install 'pyautogen[jupyter-executor]'
!pip install "pyautogen[retrievechat]"

  Using cached ipykernel-6.29.4-py3-none-any.whl.metadata (6.3 kB)
Using cached ipykernel-6.29.4-py3-none-any.whl (117 kB)
  Attempting uninstall: ipykernel
    Found existing installation: ipykernel 6.28.0
ERROR: Cannot uninstall ipykernel 6.28.0, RECORD file not found. You might be able to recover from this via: 'pip install --force-reinstall --no-deps ipykernel==6.28.0'.
  Using cached chromadb-0.5.3-py3-none-any.whl.metadata (6.8 kB)
  Using cached markdownify-0.12.1-py3-none-any.whl.metadata (8.0 kB)
  Using cached protobuf-4.25.3-cp37-abi3-manylinux2014_x86_64.whl.metadata (541 bytes)
  Using cached sentence_transformers-3.0.1-py3-none-any.whl.metadata (10 kB)
  Using cached build-1.2.1-py3-none-any.whl.metadata (4.3 kB)
  Using cached chroma_hnswlib-0.7.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (252 bytes)
  Using cached posthog-3.5.0-py2.py3-none-any.whl.metadata (2.0 kB)
  Using cached onnxruntime-1.18.0-cp310-cp310-manylinux_2_27_x86_64.manylinux_2

In [26]:
import shutil
import sys
import json
import autogen
import os
from autogen import Agent,AssistantAgent, UserProxyAgent,ConversableAgent
from autogen.coding import LocalCommandLineCodeExecutor,CodeBlock
from autogen.coding.jupyter import JupyterCodeExecutor, LocalJupyterServer
from autogen.agentchat.contrib.retrieve_assistant_agent import RetrieveAssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
from autogen.agentchat.contrib.capabilities import transform_messages, transforms
from autogen.agentchat.contrib.capabilities.transforms import  MessageHistoryLimiter, MessageTokenLimiter

# Titan-Analysis
!git clone https://github.com/Zhuang-Zhuang-Liu/Titan-Analysis.git
current_directory = os.getcwd()
sys.path.append( base_path+'/Titan-Analysis') 
from utils.utils import show_images_in_directory,folder_clean  # utils
from utils.utils import generate_guide  # rag
from utils.utils import generate_random_data,data_info_put  # IO Data
from agent_zoo.GroupChat import agent_create #agent

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


fatal: destination path 'Titan-Analysis' already exists and is not an empty directory.


In [27]:
def custom_speaker_selection_func(last_speaker: Agent, groupchat: autogen.GroupChat):
    """
    Define: a customized speaker selection function.
    Returns: Return an `Agent` class or a string from ['auto', 'manual', 'random', 'round_robin',None] to select a default method to use.
    """
    messages = groupchat.messages
    #early stop
    error_count = sum(1 for msg in messages if "(execution failed)" in msg["content"] or "FileNotFoundError" in msg["content"] )
    if error_count >= 2:
        print('当前已累计报错2次，请重新描述问题试试吧')
        return None
    
    if last_speaker is ragproxyagent:
        return classify_agent

    if last_speaker is classify_agent :
        if  "表信息查询" not in messages[-1]["content"] :
            return planner
        else:
            return analyst
        
    if last_speaker is planner:
        return project_manager
    
    if last_speaker is project_manager:
        if  "计划已完成" not in messages[-1]["content"] :
            return code_writer_agent
        else:
            return analyst
    
    if last_speaker is code_writer_agent:
        if "Code has been executed successfully" not in messages[-1]["content"] :
            return code_executor_agent
        else:
            return project_manager

    if last_speaker is code_executor_agent:
            return code_writer_agent

    if last_speaker is analyst :
        return checker    
    else:
        return None # None=终止对话
    
print('***speaker_selection_func ready***')

***speaker_selection_func ready***


In [28]:
#rag
generate_guide()
guide_path = base_path + '/data_analysis_guide.txt'

#prompt
prompt_path=base_path+'/Titan-Analysis/agent_zoo/agent_prompts.json'
with open(prompt_path, "r") as file:
    loaded_data = json.load(file)
print('***prompt ready***') # loaded_data['promopt_rag_classify_agent']

#api
api_zz_deep,url_deep,model_deep = 'sk-ca2eb7e5374c47419bc8f6276f90cda1','https://api.deepseek.com/v1',"deepseek-chat"
llm_config_deep = {"config_list": [{ "model": model_deep,"base_url": url_deep,"api_key": api_zz_deep,"temperature": 0,"cache_seed":None  }] }  

***prompt ready***


In [ ]:
#folder path
path="coding"
folder_clean(path)
task_info="""{任务} ={我想知道,销售收入大于1万、销售成本小于700的用户，主要分布在哪些年龄段和哪些等级城市，先单独分析、然后交叉分析这2个维度
                       备注： 年龄段的划分：18~30岁=青年，30~60=中年，60岁以上=老年}"""
print('***请输入需要分析的数据集地址，如需使用默认数据集，请输入“默认”***')

data_info = data_info_put(show_data_info=False,data_path=input())
print('\n***Agent Launch***\n')

user_proxy,code_writer_agent,code_executor_agent,checker,project_manager,planner,analyst,ragproxyagent,\
classify_agent = agent_create(path=path,llm_config=llm_config_deep,loaded_data=loaded_data,guide_path=guide_path)
print('\n***Analysis Start***\n')

groupchat = autogen.GroupChat(agents=[user_proxy,code_writer_agent,code_executor_agent, checker,project_manager,planner,
                                      analyst,ragproxyagent,classify_agent],
                              messages=[],max_round=27,speaker_selection_method=custom_speaker_selection_func)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config_deep)
ragproxyagent.initiate_chat(manager,message=ragproxyagent.message_generator,
                            problem = task_info + data_info ) 
show_images_in_directory(path)

***folder clean ready***
***请输入需要分析的数据集地址，如需使用默认数据集，请输入“默认”***


***sample data ready***

{数据格式示例}={
   cust_id  cust_sex  cust_age cust_revenue_type cust_open_date  cust_city_type item_item_platform item_type  item_is_discounted  item_price  item_rating  order_amt  order_cost    biz_date
0   750033         1        30                 A     2023-02-03               3          platform2     type3               False         404            4     7200.0         411  2024-05-22
1   842324         0        30                 A     2023-04-19               1          platform2     type2                True          96            5    14400.0         109  2024-03-22
2   222003         0        29                 B     2023-05-14               4          platform3     type3               False         300            1    17400.0         765  2024-05-14}

{数据地址}=/kaggle/working/Agent_DA_dateset.csv}

{数据描述}={表名：每日销售订单信息表
                        表主键：cust_id、biz_date
                        字段描述={
                            cust_id：数据类型为整数，数据格式为 6 位整数，业务含义是用户

2024-06-23 10:07:58,613 - autogen.agentchat.contrib.retrieve_user_proxy_agent - INFO - Use the existing collection `autogen-docs`.
2024-06-23 10:07:58,616 - autogen.agentchat.contrib.retrieve_user_proxy_agent - INFO - Found 1 chunks.


***agent_ready***

***Analysis Start***

Trying to create collection.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

VectorDB returns doc_ids:  [['675f24fe']]
Adding content of doc 675f24fe to context.
ragproxyagent (to chat_manager):

You're a retrieve augmented chatbot. You answer user's questions based on your own knowledge and the
context provided by the user.
If you can't answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.
You must give as short an answer as possible.

User's question is: {任务} ={我想知道,销售收入大于1万、销售成本小于700的用户，主要分布在哪些年龄段和哪些等级城市，先单独分析、然后交叉分析这2个维度
                       备注： 年龄段的划分：18~30岁=青年，30~60=中年，60岁以上=老年}
{数据格式示例}={
   cust_id  cust_sex  cust_age cust_revenue_type cust_open_date  cust_city_type item_item_platform item_type  item_is_discounted  item_price  item_rating  order_amt  order_cost    biz_date
0   750033         1        30                 A     2023-02-03               3          platform2     type3               False         404            4     7200.0         411  2024-05-22
1   842324         0        30                 A

In [18]:
def clean_library_folder(library_path):
    library_path = "/kaggle/working/Titan-Analysis"  
    try:
        shutil.rmtree(library_path)
        print(f"文件夹 {library_path} 已成功删除")
    except OSError as e:
        print(f"删除文件夹 {library_path} 时出错: {e}")
 
clean_library_folder(library_path="/kaggle/working/Titan-Analysis")

文件夹 /kaggle/working/Titan-Analysis 已成功删除
